## NTUEE儀表板資料處理中心 - 祝您好孕
##### 作者：NTUEE儀表板資料組

#### 引入必要套件與函數

In [5]:
%%capture 
!pip3 install pandas
!pip3 install geocoder
!pip3 install geopandas
!pip3 install shapely

In [6]:
%%capture
import requests
import pandas as pd
import geocoder
import geopandas as gpd
from shapely.geometry import Point, Polygon

##### 獲取臺北大數據中心資料：```get_datataipei_api```


In [7]:
def get_datataipei_api(rid):
    url = f"""https://data.taipei/api/v1/dataset/{rid}?scope=resourceAquire"""
    response = requests.request("GET", url)
    data_dict = response.json()
    count = data_dict['result']['count']
    res = list()
    offset_count = int(count/1000)
    for i in range(offset_count+1):
        i = i* 1000
        url = f"""https://data.taipei/api/v1/dataset/{rid}?scope=resourceAquire&offset={i}&limit=1000"""
        response = requests.request("GET", url)
        get_json = response.json()
        res.extend(get_json['result']['results'])
    return pd.DataFrame(res)

##### 資料庫（不斷更新！）


In [8]:
祝您好孕資料庫 = {
    "112上半年婦幼安全警示地點": "016bf772-71bd-4500-aba2-22387a1d0ae7"
}

### 祝您好孕

##### 點子
1. 婦幼安全警示區分佈圖

##### 婦幼安全警示區分佈圖

In [9]:
# 取得資料
婦幼安全警示區資料 = get_datataipei_api(祝您好孕資料庫["112上半年婦幼安全警示地點"])
print("這個資料總共有", 婦幼安全警示區資料.shape[0], "筆！")
婦幼安全警示區資料.head(10)

這個資料總共有 20 筆！


,_id,_importdate,編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註
0,1,"{'date': '2023-08-01 01:07:37.465906', 'timezo...",1\t臺北市大同區捷運中山站\t大同分局\t103\t續列
1,2,"{'date': '2023-08-01 01:07:37.475635', 'timezo...",2\t臺北市萬華區峨嵋街、漢中街口(捷運西門站)\t萬華分局\t108\t續列
2,3,"{'date': '2023-08-01 01:07:37.477156', 'timezo...",3\t臺北市萬華區西園路、和平西路口(捷運龍山寺站)\t萬華分局\t108\t續列
3,4,"{'date': '2023-08-01 01:07:37.480722', 'timezo...",4\t臺北市中山區捷運松江南京站\t中山分局\t104\t續列
4,5,"{'date': '2023-08-01 01:07:37.482010', 'timezo...",5\t臺北市中山區捷運中山國小站\t中山分局\t104\t續列
5,6,"{'date': '2023-08-01 01:07:37.483350', 'timezo...",6\t臺北市大安區捷運忠孝復興站\t大安分局\t106\t續列
6,7,"{'date': '2023-08-01 01:07:37.486254', 'timezo...",7\t臺北市中正區捷運臺北車站\t中正第一分局\t100\t續列
7,8,"{'date': '2023-08-01 01:07:37.487843', 'timezo...",8\t臺北市中正區二二八公園\t中正第一分局\t100\t續列
8,9,"{'date': '2023-08-01 01:07:37.489231', 'timezo...",9\t臺北市中正區捷運古亭站\t中正第二分局\t100\t續列
9,10,"{'date': '2023-08-01 01:07:37.490614', 'timezo...",10\t臺北市松山區新東街9巷\t松山分局\t105\t續列


In [ ]:
# 整理一下資料
婦幼安全警示區資料['地點位置'] = 婦幼安全警示區資料['編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註'].str.split('\t').str[1]

In [129]:

# 要跑3分鐘...
# 有做了一些優化，原本可是要跑十分鐘呢🔟😤
老人安養機構資料['geometry'] = 老人安養機構資料['地址'].apply(
    lambda x: (
        lambda a: Point(a.json['lng'], a.json['lat']) if a and a.json else None
    )(geocoder.arcgis(x))
)
# 沒查到經緯度的話...郭囉 誰叫你選那麼奇怪的址
老人安養機構資料.dropna(subset=['geometry'], inplace=True)


In [135]:

老人安養機構資料 = gpd.GeoDataFrame(
    老人安養機構資料,
	geometry='geometry', 
    crs='EPSG:4326'
)
老人安養機構資料 = 老人安養機構資料.to_crs('EPSG:3826') # TWD97



,_id,_importdate,屬性,機構名稱,區域別,地址,電話,收容對象,核定總床位數量,長照床位數量,養護床位數量,失智床位數量,安養床位數量,經緯度,geometry
0,1,"{'date': '2023-07-06 19:00:56.960295', 'timezo...",私立,臺北市私立上美老人長期照顧中心(養護型),士林區,臺北市士林區重慶北路4段162號1-4樓,(02)28122366,養護,33,0,33,0,0,POINT (121.51216995972777 25.084109997881626),POINT (301663.627 2775191.688)
1,2,"{'date': '2023-07-06 19:00:56.971035', 'timezo...",私立,臺北市私立璞園老人長期照顧中心(養護型),士林區,臺北市士林區富光里010鄰葫蘆街33號4、5、6樓,(02)28163696,養護,46,0,46,0,0,POINT (121.50972001324863 25.079970008265207),POINT (301418.221 2774732.184)
2,3,"{'date': '2023-07-06 19:00:56.972223', 'timezo...",私立,臺北市私立祐心老人長期照顧中心(養護型),士林區,臺北市士林區名山里 009鄰至誠路1段305巷3弄14號1-2樓,(02)28310432,養護,16,0,16,0,0,POINT (121.53172996897797 25.101450018166304),POINT (303629.161 2777119.998)
3,4,"{'date': '2023-07-06 19:00:56.973572', 'timezo...",私立,臺北市私立永青老人長期照顧中心(養護型),士林區,臺北市士林區天壽里003鄰中山北路6段427巷8號1-4樓,(02)28761908,養護、長照,31,13,18,0,0,POINT (121.52653997835074 25.114240006131226),POINT (303100.176 2778534.649)
4,5,"{'date': '2023-07-06 19:00:56.974947', 'timezo...",私立,臺北市私立祇福老人養護所,士林區,臺北市士林區三玉里016鄰忠誠路2段10巷11、13號1-2樓,(02)28317713,養護,39,0,39,0,0,POINT (121.5295279198231 25.108829068773687),POINT (303403.858 2777936.477)
5,6,"{'date': '2023-07-06 19:00:56.976219', 'timezo...",私立,臺北市私立荷園老人長期照顧中心(養護型),士林區,臺北市士林區富光里010鄰葫蘆街33號2-3樓,(02)28163696,養護,39,0,39,0,0,POINT (121.50972001324863 25.079970008265207),POINT (301418.221 2774732.184)
6,7,"{'date': '2023-07-06 19:00:56.977645', 'timezo...",私立,臺北市私立柏安老人養護所,士林區,臺北市士林區蘭雅里 024鄰 中山北路6段290巷52、54號1樓,(02)28333536,養護,20,0,20,0,0,POINT (121.52643000778113 25.109449999923555),POINT (303091.154 2778004.033)
7,8,"{'date': '2023-07-06 19:00:56.979220', 'timezo...",私立,臺北市私立松園老人長期照顧中心(養護型),士林區,臺北市士林區蘭雅里 020鄰德行東路61巷1、2、3號1-2樓,(02)88662097,養護,49,0,49,0,0,POINT (121.52646001499448 25.107599988164196),POINT (303094.979 2777799.126)
8,9,"{'date': '2023-07-06 19:00:56.980526', 'timezo...",私立,臺北市私立天玉老人長期照顧中心(長期照護型),士林區,臺北市士林區天玉街38巷18弄18號1-6樓,(02)28720239,長照,39,39,0,0,0,POINT (121.53033002350958 25.12016999308028),POINT (303479.818 2779192.989)
9,10,"{'date': '2023-07-06 19:00:56.981839', 'timezo...",私立,臺北市私立慧光老人養護所,大同區,臺北市大同區星明里 014鄰寧夏路32 號5樓,(02)25582275,養護,40,0,40,0,0,POINT (121.51480003330613 25.054690020650014),POINT (301941.339 2771933.971)


In [142]:
def create_buffer_with_opacity(center, radius, opacity):
    circle = Point(center).buffer(radius)
    return circle.exterior.buffer(0.1, cap_style=3).difference(circle)

buffer半徑 = 10000.0


# Set opacity based on the central value (核定總床位數量)

老人安養機構資料['核定總床位數量'] = 老人安養機構資料['核定總床位數量'].astype(int)
最大床位數量 = 老人安養機構資料['核定總床位數量'].max()
不透明度梯度串列 = 老人安養機構資料['核定總床位數量'].apply(lambda x: 1.0 if x == 最大床位數量 else x / 最大床位數量)

# Create a colormap with varying alpha values
著色串列 = [(1, 0, 0, alpha) for alpha in 不透明度梯度串列]  # Red color, adjust as needed

老人安養機構資料['buffers'] = 老人安養機構資料.apply(lambda row: create_buffer_with_opacity(row['geometry'].coords[0], buffer半徑 , row['核定總床位數量'] / 最大床位數量), axis=1)
老人安養機構資料['buffers']

0    POLYGON ((311663.727 2775191.683, 311615.574 2...
1    POLYGON ((311418.321 2774732.180, 311370.168 2...
2    POLYGON ((313629.261 2777119.993, 313581.108 2...
3    POLYGON ((313100.276 2778534.644, 313052.123 2...
4    POLYGON ((313403.958 2777936.472, 313355.805 2...
                           ...                        
0    POLYGON ((316753.289 2764410.965, 316705.136 2...
1    POLYGON ((307171.815 2778966.957, 307123.662 2...
0    POLYGON ((314766.709 2777187.653, 314718.556 2...
1    POLYGON ((315203.706 2765891.117, 315155.553 2...
2    POLYGON ((318987.179 2764708.399, 318939.026 2...
Name: buffers, Length: 95, dtype: geometry

### 2. 臺北歡迎您

### 3. 宜居臺北

### 4. 祝您好孕

### 5. 韌性城市